In [1]:
# default_exp signUp

# SignUp

> Interface to sign up user

In [2]:
#hide
from nbdev.showdoc import *
import src.setEnv 

In [3]:
#export 
from src.userTable import UserTable
from awsSchema.apigateway import Response, Event
from copy import deepcopy
from beartype import beartype
from uuid import uuid4
from src.saltHashPassword import hash_password, check_password
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from time import time
import json

In [4]:
#check whether hash password is working here
examplePassword1 = '123'
exampleHashedPassword1 = hash_password(examplePassword1)
exampleHashedPassword1

'ad5a3de48f4a7bd0cc04f70f0fafc96af1301fcf1cb6e562d745c546ee9562d3:f191780be98f4cd49846a10582af7fbf'

## User Input Class

In [5]:
#export 
@dataclass_json
@dataclass
class UserInput:
    username: str
    password: str
    email: str 
    
    @property
    def passwordHash(self):
        return hash_password(self.password)
    
    def saveTable(self):
        passwordHashed = self.passwordHash
        userTable = UserTable(
                userId = str(uuid4()),
                username = self.username,
                date = time(),
                passwordHash = passwordHashed,
                email = self.email
            )
        userTable.save()
        

In [7]:
example = UserInput.from_dict({'username': 'DelG', 'password': '123', 'email': 'chanopsethiwan233@gmail.com'})
example

UserInput(username='DelG', password='123', email='chanopsethiwan233@gmail.com')

## Main Lambda Handler

In [8]:
#export 
class H:
    class ParseInputError(Exception): pass
    class SavingError(Exception): pass
    class UsernameAlreadyExistError(Exception): pass
    @classmethod
    @beartype
    def parseInput(cls, event:dict)->UserInput:
        try:
            user = Event.parseDataClass(UserInput, deepcopy(event))
            return user
        except Exception as e:
            raise cls.ParseInputError(e)
    
    @classmethod
    @beartype
    def saveUserMethod(cls, user:UserInput)->bool:
        try:
            user.saveTable()
            return True
        except Exception as e:
            raise cls.SavingError(e)
    
        

In [9]:
#export
def signUp(event, *args):
    try:
        user = H.parseInput(event)
        H.saveUserMethod(user)
        return Response.returnSuccess()
    except H.ParseInputError as e:
        return Response.returnError(f'failed to parse input {e}')
    except H.SavingError as e:
        return Response.returnError(f'failed saving user {e}')
    except Exception as e:
        return Response.returnError(f' unknown error {e}')
    

In [10]:
input_ = {'username': 'Delgg4', 'password': '12765', 'email': 'chanopsethiwan233@gmail.com'}
event = Event.getInput(body = input_)
signUp(event)

{'body': '{}',
 'statusCode': 200,
 'headers': {'Access-Control-Allow-Headers': '*',
  'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Methods': '*'}}

In [11]:
epoch = time()
epoch

1660748952.9055529

In [12]:
a = Event.getInput({'username': 'DelG197', 'password': '123456'})
json.dumps(a)

'{"body": "{\\"username\\":\\"DelG197\\",\\"password\\":\\"123456\\"}", "httpMethod": null, "multiValueHeaders": null, "multiValueQueryStringParameters": null, "path": null, "pathParameters": null, "queryStringParameters": {}, "requestContext": null, "headers": {}, "statusCode": 200, "isBase64Encoded": false}'

In [13]:
a = 'very interesting'
if a:
    print('true')
else:
    print('false')

true
